In [1]:
# 套件與初始化
import pandas as pd
import requests
import urllib.parse
import os
import warnings
warnings.filterwarnings('ignore')

UNIT_ID_LIST = {113: "蔬菜", 135: "果品"}
YEAR_CUTOFF = 2024 # 起始年份
# 以下填入需要查詢的水果(不支援模糊查詢)
# VEGETABLES = ['蘋果', '桃子', '葡萄', '香蕉', '芒果', '木瓜', '鳳梨', '草莓', '藍莓', '覆盆子']
VEGETABLES = ['蘋果']

os.makedirs('蔬果資料', exist_ok=True)
print(f"共 {len(VEGETABLES)} 種蔬果，準備下載...")

共 1 種蔬果，準備下載...


In [2]:
import urllib3
import requests
import pandas as pd
import urllib.parse

# 停用 SSL 警告
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

# 建立一個空的Dataframe，來存放所有資料
df_sum = pd.DataFrame()

# 抓取並儲存每一種蔬果的資料
# for name in VEGETABLES:
#     try:
#         for id, kind in UNIT_ID_LIST.items():
#             filter_str = f"年度 gt {YEAR_CUTOFF} and {kind}類別 like {name}"
#             encoded = urllib.parse.quote(filter_str, safe='')
       
#             url = f"https://data.moa.gov.tw/Service/OpenData/DataFileService.aspx?UnitId={id}&$top=1000&$skip=0&$filter={encoded}"

#             # 加入 verify=False 停用 SSL 驗證
#             r = requests.get(url, timeout=30, verify=False)
#             data = r.json()
            
#             if data:
#                 df = pd.DataFrame(data)
#                 fname = f"蔬果資料/{YEAR_CUTOFF}-2024年{name}.csv"
#                 if kind == "蔬菜":
#                     df = df.rename(columns={"種植面積":"種植面積_公頃", "收穫面積":"收穫面積_公頃"}) # 針對蔬菜欄位的名稱要做修改
#                 df = df[["年度", "地區別", f"{kind}類別", "種植面積_公頃", "收穫面積_公頃"]]
#                 df = df.rename(columns={f"{kind}類別": "蔬果類別"}) # 統一欄位名稱
#                 df = df[df["蔬果類別"] == name] # 只篩選需要的名稱，避免不同水果被抓進來(ex:要抓葡萄，但葡萄柚也被抓進來)
#                 df_sum = pd.concat([df_sum, df]) # 合併資料表進入df_sum
#                 df.to_csv(fname, index=False, encoding='utf-8-sig')
#                 print(f"✅ 已儲存: {fname} ({len(df)} 筆)")
#             else:
#                 print(f"⚠️ 無資料: {name}")
#     except Exception as e:
#         print(f"❌ 發生錯誤：{name} → {e}")


# 迭代所有資料來源
for id, kind in UNIT_ID_LIST.items():

    # 初始化分頁參數
    TOP_COUNT = 1000  # 每頁抓取筆數
    SKIP_COUNT = 0    # 從第 0 筆開始
    has_more_data = True

    current_unit_df = pd.DataFrame()  # 暫存這個 UnitId 抓到的所有資料

    print(f"--- 開始抓取 {kind} 資料 (UnitId: {id}) ---")

    while has_more_data:
        try:
            filter_str = f"年度 gt {YEAR_CUTOFF}"
            encoded = urllib.parse.quote(filter_str, safe='')

            # 修正後的 URL：動態替換 $top 和 $skip
            url = (
                f"https://data.moa.gov.tw/Service/OpenData/DataFileService.aspx?"
                f"UnitId={id}&$top={TOP_COUNT}&$skip={SKIP_COUNT}&$filter={encoded}"
            )

            r = requests.get(url, timeout=30, verify=False)
            data = r.json()

            if data:
                df = pd.DataFrame(data)
                rows_fetched = len(df)

                print(
                    f"    ✅ 成功抓取 {kind} 資料：$skip={SKIP_COUNT}, 筆數: {rows_fetched}")

                # 處理欄位名稱
                if kind == "蔬菜":
                    df = df.rename(
                        columns={"種植面積": "種植面積_公頃", "收穫面積": "收穫面積_公頃"})

                df = df[["年度", "地區別", f"{kind}類別", "種植面積_公頃", "收穫面積_公頃"]]
                df = df.rename(columns={f"{kind}類別": "蔬果類別"})

                current_unit_df = pd.concat([current_unit_df, df])

                # 判斷是否還有下一頁資料
                if rows_fetched < TOP_COUNT:
                    # 抓取的筆數小於 $top，表示已經是最後一頁
                    has_more_data = False
                else:
                    # 抓取的筆數等於 $top，表示可能還有下一頁，更新 $skip 參數
                    SKIP_COUNT += TOP_COUNT

            else:
                # API 回傳空列表，表示結束
                print(f"    ⚠️ {kind} 資料 ($skip={SKIP_COUNT}) 無數據或已結束。")
                has_more_data = False

        except Exception as e:
            print(f"❌ 發生錯誤（{kind}, $skip={SKIP_COUNT}）：{e}")
            has_more_data = False  # 出錯時停止迴圈

# 將這個 UnitId 抓到的所有資料合併到總資料框中
    df_sum = pd.concat([df_sum, current_unit_df])
    print(f"--- {kind} 資料抓取完成，總筆數: {len(current_unit_df)} ---\n")


if len(df_sum.index) > 0:
    df_sum = df_sum.sort_values(by=['年度', '蔬果類別', '地區別'], ascending=False)
    df_sum.to_csv(f"蔬果資料/{YEAR_CUTOFF}-2024年彙整.csv", index=False, encoding='utf-8-sig')

--- 開始抓取 蔬菜 資料 (UnitId: 113) ---
    ✅ 成功抓取 蔬菜 資料：$skip=0, 筆數: 902
--- 蔬菜 資料抓取完成，總筆數: 902 ---

--- 開始抓取 果品 資料 (UnitId: 135) ---
    ✅ 成功抓取 果品 資料：$skip=0, 筆數: 770
--- 果品 資料抓取完成，總筆數: 770 ---

